# Preprocessing
- Drop redundancies
- Handing missing values
- Encoding

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import pandas as pd
dataset = pd.read_excel('pharma.xlsx')

In [ ]:
dataset.info()

In [ ]:
# dataset.head()

In [ ]:
columns = dataset.columns
columns

In [ ]:
for column in columns:
    print(f'{column}: {dataset[column].nunique()}')

In [ ]:
top = dataset['Tỉnh'].value_counts()
top.head(20)

## Drop redundancies
- Drop columns with vast categories (numbers of unique)

### Case 1
- mã đơn hàng, mã kh, tên kh, tên nhân viên: drop vì
    - nunique lớn (nhiều thể loại)
    - 
- Dơn giá không liên quan vì = số lượng x giá (giá có thể thay đổi nhưng drop để đơn giản trong TH này)
- Tên thuốc dễ nhập sai -> giữ mã thuốc, bỏ tên thuốc, đơn vị
- địa điểm: chi nhánh, Tỉnh, quận/huyện, xã/phường -> chọn tỉnh (drop còn lại)

In [ ]:
drop_list = ['Mã đơn hàng', 'Mã KH', 'Tên KH', 'Xã/phường', 'Quận/huyện', 'Tên nhân viên', 'Tên thuốc', 'Đơn vị', 'Đơn giá', 'Chi nhánh' ]
df1 = dataset.drop(drop_list, axis=1)
df1.head()

In [ ]:
df1.info()

### Choose top 7 popular city:

In [ ]:
value_counts = df1['Tỉnh'].value_counts()
value_counts.head(7)

In [ ]:
# Get the top 7 values
top_values = value_counts.nlargest(7).index.tolist()

# Create a boolean mask
mask = df1['Tỉnh'].isin(top_values)

# Select the rows that contain the top 7 values
df_city = df1.loc[mask]
df_city

In [ ]:
df_city.info()

### Select top popular drugs (8 in temporatory)

In [ ]:
drug_50 = df_city['Mã thuốc'].value_counts()
top_50_drug = drug_50.head(50)
# top_50_drug = drug_50

for code, value in zip(list(top_50_drug.index), list(top_50_drug.values)):
    entry = dataset.loc[dataset['Mã thuốc'] == code].iloc[0]
    drug_name = entry['Tên thuốc']
    print(f'Drug code: {code}, Name drug: {drug_name}, Count: {value}')
    # print(f'{code},{drug_name},{value}')

In [ ]:
# in future will replace by 8 actual drugs
drug_columns = list(top_50_drug.head(8).index)
drug_columns

In [ ]:
# Keep drup in drug_columns and drop others

# Create a boolean mask
mask = df_city['Mã thuốc'].isin(drug_columns)

# Dropping
df_city = df_city.loc[mask]
df_city

In [ ]:
for column in df_city.columns:
    print(f'{column}: {df_city[column].nunique()}')

**Visualize amount of drug**
-  To reduce the vast number of few numbers in or to reduce
over-fitting, data were processed before model training using log transformation methods.
Quantity of the amount utilized were changed to float type. The modified numbers were
returned to normal using exponential power, according to the prediction

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# get the 'Số lượng' column
amount = df_city['Số lượng']

# Add a small constant to avoid log(0)
amount = amount + 1e-10

# Filter out negative values
amount = amount[amount > 0]

log_amount = np.log(amount)
log_amount.plot(kind='hist', bins=100, figsize = (8,6))
plt.show()

In [ ]:
# Filter out any negative values (outliers)
log_amount = log_amount[log_amount > 0.1]
log_amount.plot(kind='hist', bins=100, figsize = (8,6))
plt.show()

- Drop outliers: frequency < 10^0.1

In [ ]:
# Log transform the 'income' column
df_city['Số lượng'] = np.log(df_city['Số lượng'])

# Define a threshold value
threshold = 0.1

# Drop any rows where the log-transformed 'Số lượng' value is less than the threshold
df_city.drop(df_city[df_city['Số lượng'] < threshold].index, inplace=True)
df_city.head()

In [ ]:
# Save as csv for easy manipulate
df_city.to_csv('pre1_pharma.csv', index=False)

# Choosing model

### Encoding
- Time-feature
- One hot: Tỉnh, Mã

In [ ]:
# Import data after log and preprocessing 1
data_for_encode = pd.read_csv('pre1_pharma.csv', index_col='Ngày')
data_for_encode

### Test-train splitting

In [ ]:
from sklearn.model_selection import train_test_split

# Separate target from predictors
y = data_for_encode['Số lượng']
X = data_for_encode.drop(['Số lượng'], axis=1)

In [ ]:
# Divide data into training and validation subsets
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

## Linear regression

## Random Forest

## ANN